# Automated spike sorting, quality metrics, and data alignment. 
This is a pipeline which spike-sorts neuropixel data aquired with [SpikeGLX](https://billkarsh.github.io/SpikeGLX/) and task control via either [NIMH MonkeyLogic](https://monkeylogic.nimh.nih.gov/index.html) or [PsychToolbox](https://psychtoolbox.org/). 

It uses [Kilosort 4](https://github.com/MouseLand/Kilosort) and KS4 must already be installed on this machine.

This program will:
- preprocess your spiking data (e.g. correcting for multiplexing error)
- spike sort your data (via Kilosort 4)
- generate basic quality metrics for each putative unit identified by KS4
- extract and resample the LFP data (data are resampled at 1000 Hz)
- extract 8 bit digital words used to mark specific [task events](https://monkeylogic.nimh.nih.gov/docs_RuntimeFunctions.html#eventmarker)
- extract [sync edges](https://billkarsh.github.io/SpikeGLX/help/syncEdges/Sync_edges/) associated with each data stream 
- map all of your data into a common timeline (the clock of your first neuropixels probe - imec0).

All data are then saved in their original directories. 

**Notes:** 
- This pipeline is optimized for data acquisition with [SpikeGLX](https://billkarsh.github.io/SpikeGLX/) where each probe has its own folder
- You must use (activate) the 'kilosort' environment you created when installing KS4
- You must use a version of numpy less than 2.0. To do this, navigate to the anaconda prompt and `uninstall numpy` and then `pip install numpy==1.24.0`

This version was created by [Thom Elston](https://www.thomelston.com/) in June 2024.

In [1]:
import kilosort as ks
import numpy as np
import pandas as pd
from pathlib import Path

import neuropixel_preprocessing_module as npm

### User variables

In [7]:
base_folder = Path('E:/D20231212_Rec03_g0/') # highest level folder for this recording
brain_areas = ['CdN', 'OFC'] # which brain area is probe 0 and 1?
run_spike_sorter = True # set to false if you want to only run alignment or skip sorting for some reason
channel_map = 'neuropixPrimate_kilosortChanMap.mat'

#### Main routines run below

In [2]:
# subfolders for each probe
probe_folders = [folder for folder in base_folder.glob('*') if folder.is_dir()]

# save csv with info about which probe was in which brain area
pd.DataFrame(brain_areas, columns=['brain_area']).to_csv(base_folder / 'brain_areas.csv')

# initialize a list to accumulate the names of the sync_files into
# - these are what are used to map data into a common timeline
sync_files = []
data_stream_info = []

In [3]:
# Kilosort 4 run settings (sample rate will be modified later for each probe)
settings = {'n_chan_bin': 385, 'fs': 30000}

# Start looping over each probe
for i in range(len(probe_folders)):

    probe_dir = probe_folders[i]
    probe_path = npm.get_path_from_dir(probe_dir, 'ap.bin')
    i_brain_area = brain_areas[i]

    print('Loading ' + str(probe_dir) + ' in ' + str(i_brain_area) + '\n')
    print('File path %s\n' % str(probe_path))

    # collect the name of the sync_files for later
    sync_files.append(npm.get_path_from_dir(probe_dir, 'lf.bin'))
    data_stream_info.append('imec'+ str(i))
    
    probe_meta = npm.readMeta(probe_path) # read in probe meta data
    settings['fs'] = float(probe_meta['imSampRate']) # set probe sample rate
    
    results_dir = probe_dir / 'ks4_out/sorter_output' # set Kilosort results directory
    results_dir.mkdir(parents=True, exist_ok=True)
    
    if run_spike_sorter:
    # run Kilosort 4
        ks.run_kilosort(settings=settings, filename=probe_path, probe_name=channel_map, results_dir=results_dir)
        print('Sorting complete!' + '\n')

        # Read in probe-specific timing info from the meta data
        fs = float(probe_meta['imSampRate']) # sampling rate, in Hz
        rec_length = float(probe_meta['fileTimeSecs']) # recording duration (in s)

        # compute some quality metrics
        print('Computing quality metrics...' + '\n')
        npm.compute_basic_quality_metrics(results_dir, fs, rec_length)

    print('Quality metrics saved\n')
    print('Probe %i complete\n\n' % i) 



Loading E:\D20231212_Rec03_g0\D20231212_Rec03_g0_imec0 in CdN

File path E:\D20231212_Rec03_g0\D20231212_Rec03_g0_imec0\D20231212_Rec03_g0_t0.imec0.ap.bin

Quality metrics saved

Probe 0 complete


Loading E:\D20231212_Rec03_g0\D20231212_Rec03_g0_imec1 in OFC

File path E:\D20231212_Rec03_g0\D20231212_Rec03_g0_imec1\D20231212_Rec03_g0_t0.imec1.ap.bin

Quality metrics saved

Probe 1 complete




Now extract the sync edges and map the spike times to a reference timeline

**NOTE:** This assumes that analog channel 0 is used as the sync data stream for the nidaq stream

In [4]:
# add the nidaq data to sync_files and now extract the edges associated with each data stream. 
sync_files.append(npm.get_path_from_dir(base_folder, 'nidq.bin'))
data_stream_info.append('nidq')
npm.extract_sync_edges(sync_files)

# extract 8 bit event words to mark task events (from e.g. MonkeyLogic or Psych Toolbox)
npm.extract_event_codes(npm.get_path_from_dir(base_folder, 'nidq.bin'))

Extracting sync edges...

n_channels: 385, n_file_samples: 8722216
n_channels: 385, n_file_samples: 8722303
n_channels: 9, n_file_samples: 36958451

Sync edge times saved in original directory.

Extracting event codes from nidq stream...

n_channels: 9, n_file_samples: 36958451

Event codes saved in original directory.


In [5]:
# map your spike and event code data into a common timeline
npm.align_data_streams(sync_files, data_stream_info, 0, ks_ver='4')

Mapping spikes and task events to a common timeline.

imec0 set as master timeline.
aligning: E:\D20231212_Rec03_g0\D20231212_Rec03_g0_imec0
aligning: E:\D20231212_Rec03_g0\D20231212_Rec03_g0_imec1
aligning: E:\D20231212_Rec03_g0

Aligned spikes and event codes saved in their original directories.



In [6]:
# extract and align the LFP streams
npm.align_LFP_streams(sync_files, data_stream_info, 0)

Extracting and mapping LFPs to a common timeline.

imec0 set as master timeline.
aligning: E:\D20231212_Rec03_g0\D20231212_Rec03_g0_imec0
n_channels: 385, n_file_samples: 8722216


E:\D20231212_Rec03_g0\D20231212_Rec03_g0_imec0: 100%|██████████| 385/385 [02:13<00:00,  2.89it/s]


aligning: E:\D20231212_Rec03_g0\D20231212_Rec03_g0_imec1
n_channels: 385, n_file_samples: 8722303


E:\D20231212_Rec03_g0\D20231212_Rec03_g0_imec1: 100%|██████████| 385/385 [02:09<00:00,  2.97it/s]



Aligned and downsampled LFPs saved in their original directories.
